# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection as ms
import harness
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA

In [2]:
pd.options.display.max_columns=None

In [3]:
def train_test_split():
    flights = pd.read_csv('data/flights_1%.csv')
    flights_train, flights_test = ms.train_test_split(
        flights, test_size=0.2, random_state=729
    )
    flights_train.to_csv('data/flights_train.csv', index_label='id')
    flights_test.to_csv('data/flights_test.csv', index_label='id')

In [4]:
#train_test_split()

In [5]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [6]:
weather_0 = pd.read_csv('weather_0.csv', index_col=0)
weather_1 = pd.read_csv('weather_1.csv', index_col=0)
weather_2 = pd.read_csv('weather_2.csv', index_col=0)
weather_3 = pd.read_csv('weather_3.csv', index_col=0)
weather_4 = pd.read_csv('weather_4.csv', index_col=0)
weather = pd.concat([weather_0,weather_1,weather_2,weather_3,weather_4])

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [7]:
flights_clean = harness.clean_train(flights_train)

In [8]:
flights_clean

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
121941,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,NaN,0,N,63,48.0,31.0,1,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109932,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,NaN,0,N,90,77.0,66.0,1,471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63108,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,NaN,0,N,137,122.0,106.0,1,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91519,2019-10-11,DL,DL_CODESHARE,DL,3798,OO,N452SW,3798,16869,XWA,"Williston, ND",13487,MSP,"Minneapolis, MN",600,609.0,9.0,31.0,640.0,817.0,3.0,806,820.0,14.0,0,NaN,0,N,126,131.0,97.0,1,553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8361,2019-09-26,UA,UA_CODESHARE,UA,5359,OO,N120SY,5359,10713,BOI,"Boise, ID",14771,SFO,"San Francisco, CA",1426,1431.0,5.0,10.0,1441.0,1504.0,7.0,1521,1511.0,-10.0,0,NaN,0,N,115,100.0,83.0,1,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48069,2019-05-01,G4,G4,G4,2170,G4,260NV,2170,14761,SFB,"Sanford, FL",11973,GPT,"Gulfport/Biloxi, MS",605,559.0,-6.0,13.0,612.0,627.0,5.0,638,632.0,-6.0,0,NaN,0,N,93,93.0,75.0,1,485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68804,2019-07-01,UA,UA_CODESHARE,UA,6259,YV,N80343,6259,10140,ABQ,"Albuquerque, NM",12266,IAH,"Houston, TX",540,553.0,13.0,12.0,605.0,853.0,3.0,851,856.0,5.0,0,NaN,0,N,131,123.0,108.0,1,744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43542,2018-07-12,AA,AA,AA,412,AA,N177US,412,11057,CLT,"Charlotte, NC",14107,PHX,"Phoenix, AZ",1300,1313.0,13.0,13.0,1326.0,1408.0,5.0,1415,1413.0,-2.0,0,NaN,0,N,255,240.0,222.0,1,1773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
flights_clean_copy = flights_clean.copy()
flights_clean_copy['month']=flights_clean_copy.fl_date
flights_clean_copy.month = flights_clean_copy.month.map(lambda v: int(v[5:7]))
flights_clean_copy['day']=flights_clean_copy.fl_date
flights_clean_copy.day = flights_clean_copy.day.map(lambda v: int(v[8:]))
flights_clean_copy['haul']=flights_clean_copy.crs_elapsed_time/60
flights_clean_copy['haul'] = pd.cut(flights_clean_copy.haul,bins=[0,3,6,12],labels=['Short','Medium','Long'])

In [10]:
flights_clean_copy

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,day,haul
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
121941,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,NaN,0,N,63,48.0,31.0,1,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,25,Short
109932,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,NaN,0,N,90,77.0,66.0,1,471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,Short
63108,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,NaN,0,N,137,122.0,106.0,1,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,Short
91519,2019-10-11,DL,DL_CODESHARE,DL,3798,OO,N452SW,3798,16869,XWA,"Williston, ND",13487,MSP,"Minneapolis, MN",600,609.0,9.0,31.0,640.0,817.0,3.0,806,820.0,14.0,0,NaN,0,N,126,131.0,97.0,1,553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,11,Short
8361,2019-09-26,UA,UA_CODESHARE,UA,5359,OO,N120SY,5359,10713,BOI,"Boise, ID",14771,SFO,"San Francisco, CA",1426,1431.0,5.0,10.0,1441.0,1504.0,7.0,1521,1511.0,-10.0,0,NaN,0,N,115,100.0,83.0,1,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,26,Short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48069,2019-05-01,G4,G4,G4,2170,G4,260NV,2170,14761,SFB,"Sanford, FL",11973,GPT,"Gulfport/Biloxi, MS",605,559.0,-6.0,13.0,612.0,627.0,5.0,638,632.0,-6.0,0,NaN,0,N,93,93.0,75.0,1,485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,Short
68804,2019-07-01,UA,UA_CODESHARE,UA,6259,YV,N80343,6259,10140,ABQ,"Albuquerque, NM",12266,IAH,"Houston, TX",540,553.0,13.0,12.0,605.0,853.0,3.0,851,856.0,5.0,0,NaN,0,N,131,123.0,108.0,1,744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1,Short
43542,2018-07-12,AA,AA,AA,412,AA,N177US,412,11057,CLT,"Charlotte, NC",14107,PHX,"Phoenix, AZ",1300,1313.0,13.0,13.0,1326.0,1408.0,5.0,1415,1413.0,-2.0,0,NaN,0,N,255,240.0,222.0,1,1773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,12,Medium


In [11]:
flights_with_weather = flights_clean_copy.merge(
    weather, left_on=['fl_date', 'origin_city_name'], right_on=['date', 'city']
).merge(
    weather, left_on=['fl_date', 'dest_city_name'], right_on=['date', 'city'],
    suffixes=('_origin', '_dest')
)

weather_category_map = {
    'Partially cloudy': 'Cloudy',
    'Clear': 'Sunny',
    'Rain, Partially cloudy': 'Rainy',
    'Rain, Overcast': 'Rainy',
    'Overcast': 'Cloudy',
    'Rain': 'Rainy',
    'Snow, Partially cloudy': 'Snowy',
    'Snow, Overcast': 'Snowy',
    'Snow': 'Snowy',
}
flights_with_weather['weather_origin'] = flights_with_weather.conditions_origin.map(weather_category_map)
flights_with_weather['weather_dest'] = flights_with_weather.conditions_dest.map(weather_category_map)

In [12]:
flights_with_weather = flights_clean_copy.merge(flights_with_weather, how='outer')

In [13]:
def add_grouped_stats(df,col):
    """
    Takes a DataFrame and a catagorical column name and adds five new columns to the Dataframe base off of grouped delay stats in relation to the catagories.
    Parameters:
        - df: The dataframe.
        - col (str): The catagoriclal column which you would like to produces stats from in relation to delay.
    Returns:
        - df_2: The new dataframe with the stat column based off of chosen col.
    """
    df_2 = df.copy()
    col_mean=col + '_delay_mean'
    col_median=col+'_delay_median'
    col_std=col+'_delay_std'
    col_min = col+'_delay_min'
    col_max=col+'_delay_max'
    df_2[col_mean] = df[col].map(df.groupby([col]).arr_delay.mean().to_dict())
    df_2[col_median] = df[col].map(df.groupby([col]).arr_delay.median().to_dict())
    df_2[col_std] = df[col].map(df.groupby([col]).arr_delay.std().to_dict())
    df_2[col_min] = df[col].map(df.groupby([col]).arr_delay.min().to_dict())
    df_2[col_max] = df[col].map(df.groupby([col]).arr_delay.max().to_dict())
    return df_2
                            


In [14]:
flights_with_weather = add_grouped_stats(flights_with_weather,'conditions_origin')
flights_with_weather = add_grouped_stats(flights_with_weather,'conditions_dest')
flights_with_weather = add_grouped_stats(flights_with_weather,'weather_origin')
flights_with_weather = add_grouped_stats(flights_with_weather,'weather_dest')
flights_with_weather = add_grouped_stats(flights_with_weather,'op_unique_carrier')
flights_with_weather = add_grouped_stats(flights_with_weather,'fl_date')
flights_with_weather = add_grouped_stats(flights_with_weather,'dest_city_name')
flights_with_weather = add_grouped_stats(flights_with_weather,'origin_airport_id')
flights_with_weather = add_grouped_stats(flights_with_weather,'origin_city_name')
flights_with_weather = add_grouped_stats(flights_with_weather,'dest_airport_id')
flights_with_weather = add_grouped_stats(flights_with_weather,'day')
flights_with_weather = add_grouped_stats(flights_with_weather,'month')
flights_with_weather = add_grouped_stats(flights_with_weather,'haul')
flights_with_weather = add_grouped_stats(flights_with_weather,'tail_num')

In [15]:
len(flights_with_weather.columns)

121

In [16]:
flights_with_weather

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,day,haul,date_origin,conditions_origin,date_dest,conditions_dest,weather_origin,weather_dest,conditions_origin_delay_mean,conditions_origin_delay_median,conditions_origin_delay_std,conditions_origin_delay_min,conditions_origin_delay_max,conditions_dest_delay_mean,conditions_dest_delay_median,conditions_dest_delay_std,conditions_dest_delay_min,conditions_dest_delay_max,weather_origin_delay_mean,weather_origin_delay_median,weather_origin_delay_std,weather_origin_delay_min,weather_origin_delay_max,weather_dest_delay_mean,weather_dest_delay_median,weather_dest_delay_std,weather_dest_delay_min,weather_dest_delay_max,op_unique_carrier_delay_mean,op_unique_carrier_delay_median,op_unique_carrier_delay_std,op_unique_carrier_delay_min,op_unique_carrier_delay_max,fl_date_delay_mean,fl_date_delay_median,fl_date_delay_std,fl_date_delay_min,fl_date_delay_max,dest_city_name_delay_mean,dest_city_name_delay_median,dest_city_name_delay_std,dest_city_name_delay_min,dest_city_name_delay_max,origin_airport_id_delay_mean,origin_airport_id_delay_median,origin_airport_id_delay_std,origin_airport_id_delay_min,origin_airport_id_delay_max,origin_city_name_delay_mean,origin_city_name_delay_median,origin_city_name_delay_std,origin_city_name_delay_min,origin_city_name_delay_max,dest_airport_id_delay_mean,dest_airport_id_delay_median,dest_airport_id_delay_std,dest_airport_id_delay_min,dest_airport_id_delay_max,day_delay_mean,day_delay_median,day_delay_std,day_delay_min,day_delay_max,month_delay_mean,month_delay_median,month_delay_std,month_delay_min,month_delay_max,haul_delay_mean,haul_delay_median,haul_delay_std,haul_delay_min,haul_delay_max,tail_num_delay_mean,tail_num_delay_median,tail_num_delay_std,tail_num_delay_min,tail_num_delay_max
0,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,NaN,0,N,63,48.0,31.0,1,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,25,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.537708,-4.0,47.071042,-59.0,674.0,-3.043689,-9.0,25.477355,-34.0,166.0,3.933333,-8.0,34.339849,-22.0,163.0,7.251644,-4.0,42.269576,-43.0,1163.0,7.251644,-4.0,42.269576,-43.0,1163.0,3.933333,-8.0,34.339849,-22.0,163.0,1.441833,-8.0,38.705541,-54.0,627.0,9.752049,-5.0,59.957840,-60.0,1417.0,5.734592,-6.0,50.170304,-59.0,1532.0,11.241379,-4.0,42.447998,-44.0,143.0
1,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,NaN,0,N,90,77.0,66.0,1,471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.509573,-5.0,31.549720,-67.0,587.0,20.691824,-5.0,114.508247,-36.0,918.0,6.174742,-6.0,58.390846,-52.0,1157.0,7.313725,-5.0,52.047747,-41.0,934.0,7.313725,-5.0,52.047747,-41.0,934.0,6.174742,-6.0,58.390846,-52.0,1157.0,6.736996,-6.0,50.716204,-57.0,918.0,3.965452,-6.0,52.066359,-67.0,1402.0,5.734592,-6.0,50.170304,-59.0,1532.0,-4.343750,-12.0,17.830039,-21.0,61.0
2,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,NaN,0,N,137,122.0,106.0,1,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [17]:


# def add_outside_grouped_stats(df, df_sub, col):
#     """
#     Creates grouped statistics of a column in One Dataframe and maps it to another subset Dataframe
#     Parameters:
#         - df: Dataframe which stats will be derived
#         - df_sub: Dataframe which will have the stats mapped onto it
#         - col (str): The catagoriclal column which you would like to produces stats from df and map to df_sub.
#     Returns:
#         - df_2: The new subset dataframe with the stat columns based on df.
#     """
#     df_2 = df_sub.copy()
#     col_mean=col + '_delay_mean'
#     col_median=col+'_delay_median'
#     col_std=col+'_delay_std'
#     col_min = col+'_delay_min'
#     col_max=col+'_delay_max'
    
#     df_2[col_mean] = df_sub[col].map(df.groupby([col]).arr_delay.mean().to_dict())
#     df_2[col_median] = df_sub[col].map(df.groupby([col]).arr_delay.median().to_dict())
#     df_2[col_std] = df_sub[col].map(df.groupby([col]).arr_delay.std().to_dict())
#     df_2[col_min] = df_sub[col].map(df.groupby([col]).arr_delay.min().to_dict())
#     df_2[col_max] = df_sub[col].map(df.groupby([col]).arr_delay.max().to_dict())
#     return df_2




# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'op_unique_carrier')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'fl_date')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'origin_city_name')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'dest_city_name')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'origin_airport_id')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'dest_airport_id')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'day')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'month')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'haul')
# flights_with_weather = add_outside_grouped_stats(flights_clean_copy, flights_with_weather, 'tail_num')

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [18]:
flights_with_weather

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,day,haul,date_origin,conditions_origin,date_dest,conditions_dest,weather_origin,weather_dest,conditions_origin_delay_mean,conditions_origin_delay_median,conditions_origin_delay_std,conditions_origin_delay_min,conditions_origin_delay_max,conditions_dest_delay_mean,conditions_dest_delay_median,conditions_dest_delay_std,conditions_dest_delay_min,conditions_dest_delay_max,weather_origin_delay_mean,weather_origin_delay_median,weather_origin_delay_std,weather_origin_delay_min,weather_origin_delay_max,weather_dest_delay_mean,weather_dest_delay_median,weather_dest_delay_std,weather_dest_delay_min,weather_dest_delay_max,op_unique_carrier_delay_mean,op_unique_carrier_delay_median,op_unique_carrier_delay_std,op_unique_carrier_delay_min,op_unique_carrier_delay_max,fl_date_delay_mean,fl_date_delay_median,fl_date_delay_std,fl_date_delay_min,fl_date_delay_max,dest_city_name_delay_mean,dest_city_name_delay_median,dest_city_name_delay_std,dest_city_name_delay_min,dest_city_name_delay_max,origin_airport_id_delay_mean,origin_airport_id_delay_median,origin_airport_id_delay_std,origin_airport_id_delay_min,origin_airport_id_delay_max,origin_city_name_delay_mean,origin_city_name_delay_median,origin_city_name_delay_std,origin_city_name_delay_min,origin_city_name_delay_max,dest_airport_id_delay_mean,dest_airport_id_delay_median,dest_airport_id_delay_std,dest_airport_id_delay_min,dest_airport_id_delay_max,day_delay_mean,day_delay_median,day_delay_std,day_delay_min,day_delay_max,month_delay_mean,month_delay_median,month_delay_std,month_delay_min,month_delay_max,haul_delay_mean,haul_delay_median,haul_delay_std,haul_delay_min,haul_delay_max,tail_num_delay_mean,tail_num_delay_median,tail_num_delay_std,tail_num_delay_min,tail_num_delay_max
0,2019-08-25,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1818,1811.0,-7.0,14.0,1825.0,1856.0,3.0,1921,1859.0,-22.0,0,NaN,0,N,63,48.0,31.0,1,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,25,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.537708,-4.0,47.071042,-59.0,674.0,-3.043689,-9.0,25.477355,-34.0,166.0,3.933333,-8.0,34.339849,-22.0,163.0,7.251644,-4.0,42.269576,-43.0,1163.0,7.251644,-4.0,42.269576,-43.0,1163.0,3.933333,-8.0,34.339849,-22.0,163.0,1.441833,-8.0,38.705541,-54.0,627.0,9.752049,-5.0,59.957840,-60.0,1417.0,5.734592,-6.0,50.170304,-59.0,1532.0,11.241379,-4.0,42.447998,-44.0,143.0
1,2018-03-02,WN,WN,WN,2028,WN,N244WN,2028,13495,MSY,"New Orleans, LA",10693,BNA,"Nashville, TN",745,738.0,-7.0,7.0,745.0,851.0,4.0,915,855.0,-20.0,0,NaN,0,N,90,77.0,66.0,1,471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.509573,-5.0,31.549720,-67.0,587.0,20.691824,-5.0,114.508247,-36.0,918.0,6.174742,-6.0,58.390846,-52.0,1157.0,7.313725,-5.0,52.047747,-41.0,934.0,7.313725,-5.0,52.047747,-41.0,934.0,6.174742,-6.0,58.390846,-52.0,1157.0,6.736996,-6.0,50.716204,-57.0,918.0,3.965452,-6.0,52.066359,-67.0,1402.0,5.734592,-6.0,50.170304,-59.0,1532.0,-4.343750,-12.0,17.830039,-21.0,61.0
2,2019-03-06,UA,UA,UA,545,UA,N69804,545,11292,DEN,"Denver, CO",12266,IAH,"Houston, TX",1214,1233.0,19.0,12.0,1245.0,1531.0,4.0,1531,1535.0,4.0,0,NaN,0,N,137,122.0,106.0,1,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [19]:
# dummy = pd.get_dummies(flights_with_weather['fl_date'])
# flights_with_weather = pd.concat([flights_with_weather,dummy], axis=1)
# flights_with_weather = flights_with_weather.drop('fl_date',axis=1)

# dummy = pd.get_dummies(flights_with_weather['mkt_unique_carrier'])
# flights_with_weather = pd.concat([flights_with_weather,dummy], axis=1)
# flights_with_weather = flights_with_weather.drop('mkt_unique_carrier',axis=1)


In [20]:
def dummy_maker(df,col):
        """
        A function that takes a Dataframe and a catagorical column name and returns dataframe with the column replaced by dummy variables.
        Parameters
            - df: The Dataframe
            - col (str): the column name
        Returns:
            - 
        """
        dummy = pd.get_dummies(df[col])
        df_2 = pd.concat([df,dummy], axis=1)
        df_2 = df_2.drop(col,axis=1)
        return df_2

In [25]:
#flights_clean_copy.merge(weather,left_on=['fl_date', 'origin_city_name'], right_on=['date', 'city'])

In [22]:
flights_with_weather = dummy_maker(flights_with_weather,'fl_date')
#flights_with_weather = dummy_maker(flights_with_weather,'mkt_unique_carrier')
# flights_with_weather = dummy_maker(flights_with_weather,'branded_code_share')
# flights_with_weather = dummy_maker(flights_with_weather,'mkt_carrier')
# flights_with_weather = dummy_maker(flights_with_weather,'mkt_carrier_fl_num')
# flights_with_weather = dummy_maker(flights_with_weather,'op_unique_carrier')
# flights_with_weather = dummy_maker(flights_with_weather,'tail_num')




In [23]:
flights_with_weather

,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,day,haul,date_origin,conditions_origin,date_dest,conditions_dest,weather_origin,weather_dest,conditions_origin_delay_mean,conditions_origin_delay_median,conditions_origin_delay_std,conditions_origin_delay_min,conditions_origin_delay_max,conditions_dest_delay_mean,conditions_dest_delay_median,conditions_dest_delay_std,conditions_dest_delay_min,conditions_dest_delay_max,weather_origin_delay_mean,weather_origin_delay_median,weather_origin_delay_std,weather_origin_delay_min,weather_origin_delay_max,weather_dest_delay_mean,weather_dest_delay_median,weather_dest_delay_std,weather_dest_delay_min,weather_dest_delay_max,op_unique_carrier_delay_mean,op_unique_carrier_delay_median,op_unique_carrier_delay_std,op_unique_carrier_delay_min,op_unique_carrier_delay_max,fl_date_delay_mean,fl_date_delay_median,fl_date_delay_std,fl_date_delay_min,fl_date_delay_max,dest_city_name_delay_mean,dest_city_name_delay_median,dest_city_name_delay_std,dest_city_name_delay_min,dest_city_name_delay_max,origin_airport_id_delay_mean,origin_airport_id_delay_median,origin_airport_id_delay_std,origin_airport_id_delay_min,origin_airport_id_delay_max,origin_city_name_delay_mean,origin_city_name_delay_median,origin_city_name_delay_std,origin_city_name_delay_min,origin_city_name_delay_max,dest_airport_id_delay_mean,dest_airport_id_delay_median,dest_airport_id_delay_std,dest_airport_id_delay_min,dest_airport_id_delay_max,day_delay_mean,day_delay_median,day_delay_std,day_delay_min,day_delay_max,month_delay_mean,month_delay_median,month_delay_std,month_delay_min,month_delay_max,haul_delay_mean,haul_delay_median,haul_delay_std,haul_delay_min,haul_delay_max,tail_num_delay_mean,tail_num_delay_median,tail_num_delay_std,tail_num_delay_min,tail_num_delay_max,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21,2018-01-22,2018-01-23,2018-01-24,2018-01-25,2018-01-26,2018-01-27,2018-01-28,2018-01-29,2018-01-30,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-02-05,2018-02-06,2018-02-07,2018-02-08,2018-02-09,2018-02-10,2018-02-11,2018-02-12,2018-02-13,2018-02-14,2018-02-15,2018-02-16,2018-02-17,2018-02-18,2018-02-19,2018-02-20,2018-02-21,2018-02-22,2018-02-23,2018-02-24,2018-02-25,2018-02-26,2018-02-27,2018-02-28,2018-03-01,2018-03-02,2018-03-03,2018-03-04,2018-03-05,2018-03-06,2018-03-07,2018-03-08,2018-03-09,2018-03-10,2018-03-11,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,2018-03-17,2018-03-18,2018-03-19,2018-03-20,2018-03-21,2018-03-22,2018-03-23,2018-03-24,2018-03-25,2018-03-26,2018-03-27,2018-03-28,2018-03-29,2018-03-30,2018-03-31,2018-04-01,2018-04-02,2018-04-03,2018-04-04,2018-04-05,2018-04-06,2018-04-07,2018-04-08,2018-04-09,2018-04-10,2018-04-11,2018-04-12,2018-04-13,2018-04-14,2018-04-15,2018-04-16,2018-04-17,2018-04-18,2018-04-19,2018-04-20,2018-04-21,2018-04-22,2018-04-23,2018-04-24,2018-04-25,2018-04-26,2018-04-27,2018-04-28,2018-04-29,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-05-06,2018-05-07,2018-05-08,2018-05-09,2018-05-10,2018-05-11,2018-05-12,2018-05-13,2018-05-14,2018-05-15,2018-05-16,2018-05-17,2018-05-18,2018-05-19,2018-05-20,2018-05-21,2018-05-22,2018-05-23,2018-05-24,2018-05-25,2018-05-26,2018-05-27,2018-05-28,2018-05-29,2018-05-30,2018-05-31,2018-06-01,2018-06-02,2018-06-03,2018-06-04,2018-06-05,2018-06-06

In [24]:
pca = PCA()
pca.fit(flights_with_weather)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

ValueError: could not convert string to float: 'AA'

In [ ]:
n_components = #To be det

In [ ]:
pca = PCA(n_components=n_components)
components = pca.fit_transform(flights_with_weather)

total_var = pca.explained_variance_ratio_.sum() * 100

labels = {str(i): f"PC {i+1}" for i in range(n_components)}
labels['color'] = 'Median Price'

fig = px.scatter_matrix(
    components,
    color=boston.target,
    dimensions=range(n_components),
    labels=labels,
    title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.